In [46]:
from datetime import datetime
import pandas as pd
import requests
from matplotlib import pyplot as plt
import os

from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from time import sleep

In [39]:
def remove_multiindex(df:pd.DataFrame):
	return df.droplevel(level=0,axis=1)

In [28]:

# link for extract html data
def getdata(url):
    r = requests.get(url)
    return r.text

def getdata_selenium(url):

    opts = Options()
    opts.headless = True

    assert opts.headless  # Operating in headless mode
    browser = Chrome(options=opts)
    browser.set_page_load_timeout(15)
    browser.get(url)
    txt = browser.page_source
    browser.close()
    return txt

In [ ]:



def fetch_player_stats_single_year(year: int):
    df = pd.read_html(getdata(f'https://www.basketball-reference.com/leagues/NBA_{year}_per_game.html'))[0]
    df_adv = pd.read_html(getdata(f'https://www.basketball-reference.com/leagues/NBA_{year}_advanced.html'))[0]
    df_on_off = pd.read_html(getdata(f'https://www.basketball-reference.com/leagues/NBA_{year}_advanced.html'))[0]

    df = df.drop(df[df['Player'] == 'Player'].index)

    df_adv = df_adv.drop(df_adv[df_adv['Player'] == 'Player'].index)

    df_on_off = df_on_off.drop(df_on_off[df_on_off['Player'] == 'Player'].index)

    new = pd.merge(df, df_adv, on=['Player', 'Age', 'Tm', 'Pos'])
    new = pd.merge(new,df_on_off , on=['Player', 'Age', 'Tm', 'Pos'])

    player_stats_list = ['Player', 'Pos', 'Age', 'Tm', 'G_x', 'GS', 'MP_x', 'FG', 'FGA',
                         'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA',
                         'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
                         'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%',
                         'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS',
                         'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP']

    player_stats_list_correct = ['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
                                 '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
                                 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PER',
                                 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%',
                                 'TOV%', 'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM',
                                 'VORP']

    # cuts down the columns to what is listed in player_stats_list_correct
    new = new[player_stats_list]
    new.columns = player_stats_list_correct

    # This gets rid of traded players entirely
    new = new.drop_duplicates(subset='Player', keep=False).reset_index(drop=True)

    # Fills nan's with 0
    new.fillna(value=0, inplace=True)

    # Adds a year column
    new['Year'] = year

    if not os.path.isdir(f'Data/Test Sets/{datetime.now().strftime("%Y")}'):
        os.mkdir(f'Data/Test Sets/{datetime.now().strftime("%Y")}')

    new.to_csv(f'Data/Test Sets/{datetime.now().strftime("%Y")}/{datetime.now().strftime("%Y%m%d")}_player_stats.csv')
    print(datetime.now().strftime("%m/%d/%Y"), 'Complete')

In [58]:
df_list_1 = pd.read_html(getdata_selenium(f'https://www.basketball-reference.com/players/l/lowryky01.html'))
df_list_2 = pd.read_html(getdata_selenium(f'https://www.basketball-reference.com/players/w/westbru01.html'))

In [66]:

def process_df(df:pd.DataFrame):
    # df_on_off_1 = df_list_1[12]
    # df_on_off_2 = df_list_2[12]
    df = remove_multiindex(df)
    df.drop(range(df[df.Season == 'Career'].index[0],len(df)),inplace=True)
    # df.drop(range(df[df.Season == 'Career'].index[0],len(df)),inplace=True)

    df = df[[i for i in df.columns if 'Unnamed' not in i]]

    return df


In [68]:
for i in [df_list_1,df_list_2,]:
    df = process_df(i[12])

    plt.plot('Age','PGA',data=df,)
    plt.legend()

# df_on_off_1.plot('Age','PGA')
# df_on_off_2.plot('Age','PGA')